# Notebook Details

In this notebook we will give Long term memory.

We have 2 tools:
- 1. Calculator Tools.
- 2. wikipwdia web search.

We will store old conversion to a list and pass this histoy as context of the query.

As of now the agent can answer only one query. But We will add looping concept here, So that we can interact with the agent as long as we want.

## Defining Tools

In [1]:
# import Necessary Library
import requests
import ollama
import json
import os
import re

from bs4 import BeautifulSoup

In [2]:
# --- Configuration --- #
MODEL_NAME = "llama3.1"
MEMORY_PATH = "memory_V2.json"
MAX_MEMORY = 5

In [3]:
# 1. Simple calculator tool
def calculator_tool(query):
    """A simple calculator tool."""
    try:
        # Keep only numbers, operators, parentheses, and dots
        cleaned_expr = query.split('=')[0]  # Remove anything after '='
        cleaned_expr = "".join(cleaned_expr).strip()
        return str(eval(cleaned_expr))
    except Exception as e:
        return f"Error in calculation: {e}"

In [4]:
# 2. Wikipedia search tool
def clean_text(text):
    """Remove HTML tags and excessive whitespace."""
    # If HTML is present, parse it
    soup = BeautifulSoup(text, "html.parser")
    clean = soup.get_text(separator=" ", strip=True)
    # Remove multiple spaces/newlines
    clean = re.sub(r"\s+", " ", clean)
    return clean


def web_search(query):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if "query" in data and "search" in data["query"]:
            results = []
            for item in data['query']['search']:
                # Remove HTML Tags
                clean_text = re.sub(r'<[^>]+>', '', item['snippet'])
                results.append(clean_text.strip())
            return "\n".join(results) if results else "No results found on Wikipedia."
        else:
            return "No results found."
    except Exception as e:
        return f"Error searching Wikipedia: {e}"

## Defining Memory

In [5]:
# Function to load memory from a file
def load_memory():
    if os.path.exists(MEMORY_PATH):
        with open(MEMORY_PATH, "r") as f:
            return json.load(f)
    return []

# Function to save memory to a file
def save_memory(memory):
    with open(MEMORY_PATH, "w") as f:
        json.dump(memory, f, indent=4)

## Defining Agent

In [6]:
# Loading memory
memory = load_memory()

In [7]:
def agent(user_input, memory = None, model="llama3.2:latest"):
    prompt = f"""
    You are a helpful agent with tools:
    1. CALCULATE: for math problems
    2. SEARCH: for Wikipedia searches

    Decide which tool to use based on the user input.
    Respond only with one of the following formats:
    CALCULATE: <expression>
    SEARCH: <query>
    ANSWER: <your direct answer>

    Recent conversation history (last {MAX_MEMORY} turns):
    {memory}

    User: {user_input}
    Agent:"""

    # Calling LLM
    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        # temperature=0.7
    )

    return response['message']['content'].strip()

# Main loop to interact with the agent
def main():
    #Loading memory
    memory = load_memory()
    print("🤖 Agent started. Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ").strip()
        print(f"You: {user_input}")
        # Exit condition
        if user_input.lower() in ["exit", "quit"]:
            print("💾 Saving memory and exiting...")
            save_memory(memory)
            break

        # Selecting las 5 memory
        decision = agent(user_input, memory[-MAX_MEMORY:])

        print(f"# ---- 🤖 Agent decision ---- #")
        print(decision)
        print(f"# ---- 🤖 Agent Answer ---- #")
        
        if decision.startswith("CALCULATE:"):
            expression = decision[len("CALCULATE:"):].strip()
            result = calculator_tool(expression)
            # print(f"Agent: {result}")
        elif decision.startswith("SEARCH:"):
            query = decision[len("SEARCH:"):].strip()
            result = web_search(query)
            # print(f"Agent: {result}")
        elif decision.startswith("ANSWER:"):
            result = decision[len("ANSWER:"):].strip()
            # print(f"Agent: {result}")
        else:
            result = "I couldn't understand the decision format."
        
        # Print the result
        print(f"🤖 Agent: {result}")

        # Adding to memory
        memory.append(f"You: {user_input}")
        memory.append(f"Agent: {result}")

        # Ensure memory does not exceed MAX_MEMORY
        if len(memory) > MAX_MEMORY * 2:
            memory = memory[-MAX_MEMORY * 2:]

    return


In [8]:
main()

🤖 Agent started. Type 'exit' to quit.

You: What is 15 * (3 + 2)?
# ---- 🤖 Agent decision ---- #
CALCULATE: 15 * (3 + 2) = 45
# ---- 🤖 Agent Answer ---- #
🤖 Agent: 75
You: Tell me about India
# ---- 🤖 Agent decision ---- #
SEARCH: India
# ---- 🤖 Agent Answer ---- #
🤖 Agent: India, officially the Republic of India, is a country in South Asia. It is the seventh-largest country by area; the most populous country since 2023;
Developmental Inclusive Alliance (INDIA) is a big tent multi-party political alliance of several political parties in India led by the country&#039;s largest opposition
The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world&#039;s fourth-largest economy by nominal
The Constitution of India is the supreme legal document of India, and the longest written national constitution in the world. The document lays down the
India, colloquially called Tiraṅgā (the tricolour), is a horizontal rectangular tricolour flag